# DataFrame examples for screencasts


This notebook contains the non-trivial examples used in the slides. In the long run 
we want to convert all DataFrame examples to tasks. 

The examples that are not in this notebook can be created trivially by using the 
`get_gapminder_sample` function with the correct argument and executing code from the 
slides. 

In [ ]:
import pandas as pd

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True
pd.options.plotting.backend = "plotly"

import plotly.express as px


def get_gapminder_sample(version="tiny"):
    df = px.data.gapminder()
    if version == "tiny":
        df = df[["country", "continent", "year", "lifeExp"]]
        df = df.query("country in ['Cuba', 'Spain']")
        df = df.query("year in [2002, 2007]")
    elif version == "tiny3":
        df = df[["country", "continent", "year", "lifeExp"]]
        df = df.query("country in ['Cuba', 'Spain']")
        df = df.query("year in [1997, 2002, 2007]")
    elif version == "full":
        pass
    elif version == "all_observations":
        df = df[["country", "continent", "year", "lifeExp"]]
    elif version == "income":
        df = df[["country", "year", "gdpPercap", "pop"]]
        df = df.query("country in ['Cuba', 'Spain']")
        df = df.query("year in [2002, 2007]")
    else:
        msg = f"Unknown version: {version}"
        raise ValueError(msg)

    return df.reset_index(drop=True)

# (Multi-)Indices

In [ ]:
df = get_gapminder_sample("tiny")
print(df.to_markdown(index=True))

In [ ]:
df.index

In [ ]:
df_clean = df.set_index(["country", "year"])
df_clean.index

In [ ]:
df_round_tripped = df_clean.reset_index()
df_round_tripped.index

In [ ]:
print(df_clean.loc["Cuba"])

In [ ]:
print(df_clean.loc["Cuba", "lifeExp"])

In [ ]:
print(df_clean.loc[("Cuba", 2002), "lifeExp"])

## Caveat (not on slides)

Syntax `.loc[a, b]` works for rows labelled (`a`, `b`), too. Actually takes precedence
over a column label `b`. But just show the explicit syntax `.loc[(a, b)]`.

In [ ]:
df_x = df_clean.copy()
df_x[2002] = 0
df_x.loc["Cuba", 2002]

# shift, diff

In [ ]:
df = get_gapminder_sample("tiny3").set_index(["country", "year"])
print(df.reset_index().to_markdown(index=False))

In [ ]:
print(df.shift(1).reset_index().to_markdown(index=False))

In [ ]:
df["lag_lifeExp"] = df.shift(1)["lifeExp"]
print(df[["lifeExp", "lag_lifeExp"]].reset_index().to_markdown(index=False))

In [ ]:
print(
    df.groupby("country")
    .shift(1)[["continent", "lifeExp"]]
    .reset_index()
    .to_markdown(index=False)
)

In [ ]:
# df.groupby("country").diff(1)

In [ ]:
print(df.groupby("country")[["lifeExp"]].diff(1).reset_index().to_markdown(index=False))

## Concatenating DataFrames

In [ ]:
# concat without axis argument

df = get_gapminder_sample("tiny")[["country", "year", "lifeExp"]]
top = df.query("country == 'Cuba'")
bottom = df.query("country == 'Spain'").reset_index(drop=True)

print(top.to_markdown(index=False), "\n\n", sep="")
print(bottom.to_markdown(index=False), "\n\n", sep="")
print(pd.concat([top, bottom]).to_markdown(index=False), "\n\n", sep="")

In [ ]:
# concat without axis argument, but bad index

print(top.to_markdown(index=True), "\n\n", sep="")
print(bottom.to_markdown(index=True), "\n\n", sep="")
print(pd.concat([top, bottom]).to_markdown(index=True), "\n\n", sep="")

In [ ]:
life_exp = get_gapminder_sample("tiny")[["country", "year", "lifeExp"]].set_index(
    ["country", "year"]
)["lifeExp"]
gdp_pc = get_gapminder_sample("income")[["country", "year", "gdpPercap"]].set_index(
    ["country", "year"]
)

df = pd.merge(left=life_exp, right=gdp_pc, left_index=True, right_index=True)

print(life_exp.reset_index().to_markdown(index=False), "\n\n", sep="")
print(gdp_pc.reset_index().to_markdown(index=False), "\n\n", sep="")
print(df.reset_index().to_markdown(index=False), "\n\n", sep="")

In [ ]:
life_exp = get_gapminder_sample("tiny")[["country", "year", "lifeExp"]].set_index(
    pd.Index(["A", "B", "C", "D"])
)
gdp_pc = get_gapminder_sample("income")[["country", "year", "gdpPercap"]].set_index(
    pd.Index([5, 9, 3, 1])
)

df = pd.merge(
    left=life_exp,
    right=gdp_pc,
    left_on=["country", "year"],
    right_on=["country", "year"],
)

df = pd.merge(
    left=life_exp,
    right=gdp_pc,
    on=["country", "year"],
)

print(life_exp.to_markdown(index=True), "\n\n", sep="")
print(gdp_pc.to_markdown(index=True), "\n\n", sep="")
print(df.to_markdown(index=True), "\n\n", sep="")

In [ ]:
# 1:1 merge (pandas/merging/screencast)

life_exp = (
    get_gapminder_sample("tiny")[["country", "year", "lifeExp"]]
    .set_index(["country", "year"])
    .iloc[:3]
)
gdp_pc = (
    get_gapminder_sample("income")[["country", "year", "gdpPercap"]]
    .set_index(["country", "year"])
    .iloc[1:]
)

df = pd.merge(
    left=life_exp,
    right=gdp_pc,
    on=["country", "year"],
    how="outer",
)

print(life_exp.reset_index().to_markdown(index=False), "\n\n", sep="")
print(gdp_pc.reset_index().to_markdown(index=False), "\n\n", sep="")
print(df.reset_index().to_markdown(index=False).replace("nan", "   "), "\n\n", sep="")

In [ ]:
# m:1 merge

life_exp = get_gapminder_sample("tiny")[["country", "year", "lifeExp"]].set_index(
    ["country", "year"]
)

cap = pd.DataFrame.from_dict(
    {"country": ["Cuba", "Spain"], "capital": ["Havana", "Madrid"]}
).set_index("country")

df = pd.merge(
    left=life_exp,
    right=cap,
    left_index=True,
    right_index=True,
)

print(life_exp.reset_index().to_markdown(index=False), "\n\n", sep="")
print(cap.to_markdown(index=True), "\n\n", sep="")
print(df.reset_index().to_markdown(index=False), "\n\n", sep="")

# query

In [ ]:
life_exp = get_gapminder_sample("tiny")[["country", "year", "lifeExp"]].set_index(
    ["country", "year"]
)
life_exp

In [ ]:
life_exp.query?